# Api URLS
## https://www.alphavantage.co/documentation/
## https://datahub.io/core/s-and-p-500-companies#data-cli

In [1]:
# import dependencies
import csv

# API Dependencies
import requests
import json
from pandas.io.json import json_normalize # deal with nested columns in api

# data science libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Config file keys.  We can import multiple keys variales in here
from config import av_key # api key for alpha vantage

In [61]:
# determine the top companies from datahub stock cli
datahub_df = pd.read_csv("./constituents-financials_csv.csv") # raw_df

# sector data
sector_df = datahub_df.groupby("Sector").sum() 
sub_df = sector_df.nlargest(3, 'Market Cap')
sector_df

,Price,Price/Earnings,Dividend Yield,Earnings/Share,52 Week Low,52 Week High,Market Cap,EBITDA,Price/Sales,Price/Book
Sector,,,,,,,,,,
Consumer Discretionary,10418.90,2119.49,132.082638,409.84,12342.2399,8071.758553,3213562747315,2.941510e+11,191.164584,2440.66
Consumer Staples,2711.98,746.75,82.735293,124.42,3156.2977,2343.601000,2087076388082,1.799275e+11,97.569396,1916.46
Energy,1852.40,1233.89,64.462468,-52.75,2322.8700,1540.519500,1357313712749,1.420450e+11,118.067035,73.20
Financials,6055.81,1169.53,137.172367,301.71,6923.8857,4943.223800,3442649464852,3.597660e+10,228.356354,458.00
Health Care,8083.46,1406.27,55.951842,264.77,9806.2706,6326.874430,3244359043367,2.231056e+11,294.461598,540.34
Industrials,7831.47,1440.12,99.119260,358.05,9016.8249,6086.080600,2411541173034,2.052230e+11,172.417862,605.40
Information Technology,8347.00,2602.59,85.994554,217.70,9714.5049,6432.399100,6727121800912,4.313397e+11,411.609970,761.15
Materials,2559.67,513.56,43.449399,123.30,2950.9713,2139.581200,692300259151,5.710869e+10,71.902337,119.72
Real Estate,2927.52,741.15,128.527017,78.91,3648.1650,2734.976900,625315677562,4.568073e+10,328.768488,198.16


In [43]:
# ALPHA VANTAGE API SCRAPING

# will need function, symbol, and api key in order to import the json from the api
url_alpha_vantage = "https://www.alphavantage.co/" 

# url variables
function = "TIME_SERIES_DAILY"
symbol = "MSFT"
outputsize = "full" # full or compact (100 latest vs all data)
query = url_alpha_vantage + "query?" + "function=" + function + "&symbol=" + symbol +"&outputsize=" + outputsize + "&apikey=" + av_key  
url_response = requests.get(query).json()

# pretty print the json
# print(json.dumps(url_response, indent=4, sort_keys=True))

In [51]:
# create a data frame from the json
microsoft_df = pd.DataFrame.from_dict(url_response, orient = 'columns')

# only look at data from the past 3 years 
microsoft_df = microsoft_df.tail(730)